CÁLCULO DE VIGA HIPERESTÁTICA - ANÁLISE MATRICIAL

Descrição do Problema

![Alt text](image-7.png)

In [1]:
# Montagem do problema
# .Nó1_______Elemento 1_______.Nó2_______Elemento 2__________.Nó3
#             L1 = 8m                       L2 = 5m

# 1,2                          3,4                           5,6 

Importando as Bibliotecas do Python

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Dados Elemento 1 (Nó 1-2)

In [3]:
b = 0.12 # Geometria: Base da viga, em m.
h = 0.40 # Geometria: Altura  da viga, em m.
E = 50e9 # Módulo de Elasticidade da viga, em Pa.
I = round(b*h**3/12, 5) # Inércia da Viga, em m^4.
L = 8 # Comprimento do elemento 1 da viga, em m.
w = 2e3 # Carga Distribuida aplicada no elemento 1, em N/m.

Matriz de Rigidez Local do Elemento 1

In [4]:
k1 = 12*E*I/L**3
k2 = 6*E*I/L**2
k3 = 4*E*I/L
k4 = k3/2

# Identificação     1    2    3     4
ke1 = np.array([  [ k1,  k2, -k1,  k2], # 1
                  [ k2,  k3, -k2,  k4], # 2
                  [-k1, -k2,  k1, -k2], # 3
                  [ k2,  k4, -k2,  k3], # 4
                ])

print('Matriz de Rigidez (Elemento 1): ')
print(ke1)

Matriz de Rigidez (Elemento 1): 
[[  750000.  3000000.  -750000.  3000000.]
 [ 3000000. 16000000. -3000000.  8000000.]
 [ -750000. -3000000.   750000. -3000000.]
 [ 3000000.  8000000. -3000000. 16000000.]]


Vetor de Forças Equivalentes do Elemento 1

In [5]:
f1 = w*L/2
f2 = w*L**2/12

feq1 = np.array([ [-f1],
                  [-f2],
                  [-f1],
                  [ f2], 
                ])

print('Vetor de Forças Equivalentes (Elemento 1): ')
print(feq1)

Vetor de Forças Equivalentes (Elemento 1): 
[[ -8000.        ]
 [-10666.66666667]
 [ -8000.        ]
 [ 10666.66666667]]


Dados Elemento 2 (Nó 2-3)

In [6]:
b = 0.12 # Geometria: Base da viga, em m.
h = 0.40 # Geometria: Altura  da viga, em m.
E = 50e9 # Módulo de Elasticidade da viga, em Pa.
I = round(b*h**3/12, 5) # Inércia da Viga, em m^4.
L2 = 5 # Comprimento do elemento 2 da viga, em m.

# Locação da Carga Concentrada no Elemento 2
a = 2 # em m
b = 3 # em m

# Intensidade da Carga Concentrada
P = 20e3 # em N

Matriz de Rigidez Local do Elemento 2

In [7]:
k1 = 12*E*I/L2**3
k2 = 6*E*I/L2**2
k3 = 4*E*I/L2
k4 = k3/2

# Identificação     3    4    5     6
ke2 = np.array([  [ k1,  k2, -k1,  k2],  # 3
                  [ k2,  k3, -k2,  k4],  # 4
                  [-k1, -k2,  k1, -k2],  # 5
                  [ k2,  k4, -k2,  k3],  # 6
                ])

print('Matriz de Rigidez (Elemento 2): ')
print(ke2)

Matriz de Rigidez (Elemento 2): 
[[ 3072000.  7680000. -3072000.  7680000.]
 [ 7680000. 25600000. -7680000. 12800000.]
 [-3072000. -7680000.  3072000. -7680000.]
 [ 7680000. 12800000. -7680000. 25600000.]]


Vetor de Forças Equivalentes do Elemento 2

In [8]:
f3 = P*b**2*(3*a+b)/L2**3
f4 = P*a*b**2/L2**2
f5 = P*a**2*(3*b+a)/L2**3
f6 = P*b*a**2/L2**2

feq2 = np.array([ [-f3],
                  [-f4],
                  [-f5],
                  [ f6], 
                ])

print('Vetor de Forças Equivalentes (Elemento 2): ')
print(feq2)

Vetor de Forças Equivalentes (Elemento 2): 
[[-12960.]
 [-14400.]
 [ -7040.]
 [  9600.]]


Matriz de Rigidez Global K da Estrutura

In [9]:
# Criar uma matriz 6x6 preenchida com zeros
K = np.zeros((6,6))

# Transportar a rigidez do elemento 1 para a Matriz da Estrutura
K[0:4, 0:4] = K[0:4, 0:4] + ke1[0:4,0:4]

# Transportar a rigidez do elemento 2 para a Matriz da Estrutura
K[2:6, 2:6] = K[2:6, 2:6] + ke2[0:4,0:4]

print(K)

[[  750000.          3000000.          -750000.          3000000.
         0.                0.        ]
 [ 3000000.         16000000.         -3000000.          8000000.
         0.                0.        ]
 [ -750000.         -3000000.          3822000.          4680000.
  -3072000.          7680000.        ]
 [ 3000000.          8000000.          4680000.         41600000.00000001
  -7680000.         12800000.        ]
 [       0.                0.         -3072000.         -7680000.
   3072000.         -7680000.        ]
 [       0.                0.          7680000.         12800000.
  -7680000.         25600000.        ]]


Vetor F de Esforços Externos

In [10]:
# Construir um vetor 6x1 composto por zeros
F = np.zeros((6,1))

Vetor de Esforços Feq Equivalentes da Estrutra

In [11]:
# Criar Feq preenchida com zeros
Feq = np.zeros((6,1))

# Transportar feq1 para Feq
Feq[0:4, 0] = feq1[0:4,0]

# Transportar feq2 para Feq
Feq[2:6, 0] = Feq[2:6,0] + feq2[0:4,0]

print(Feq)

[[ -8000.        ]
 [-10666.66666667]
 [-20960.        ]
 [ -3733.33333333]
 [ -7040.        ]
 [  9600.        ]]


Sistema de Equações via Matrizes Reduzidas (GDL Ativos)

In [12]:
Kred = np.array([[K[1,1], K[1,3]],[K[3,1], K[3,3]]])
print (Kred)

Fred = np.array([[F[1,0] + Feq[1,0]],[F[3,0] + Feq[3,0]]])
print (Fred)

[[16000000.          8000000.        ]
 [ 8000000.         41600000.00000001]]
[[-10666.66666667]
 [ -3733.33333333]]


Solução do Sistema de Equações via Matrizes Reduzidas (GDL Ativos)

In [13]:
dred = np.linalg.solve(Kred, Fred)

# Resultado em Radianos

print(dred)

[[-6.87943262e-04]
 [ 4.25531915e-05]]


CÁLCULO DAS REAÇÕES NOS APOIOS

Faz-se necessário executar a operação: (R = K*d - Feq) onde d é o vetor de deslocamento da estrutura.

In [14]:
# Construir Vetor de Deslocamentos completo
d = np.zeros((6,1))
d[1] = dred[0]
d[3] = dred[1]

# Cálculo das Reações (forças/momentos nos nós) como R = K*d
R = np.matmul(K,d) - Feq

print(R)

[[ 6.06382979e+03]
 [ 0.00000000e+00]
 [ 2.32229787e+04]
 [-4.54747351e-13]
 [ 6.71319149e+03]
 [-9.05531915e+03]]


![Alt text](image-8.png)

CÁLCULO DOS ESFORÇOS NOS ELEMENTOS

Faz-se necessário executar a operação: (fe = Ke*de - Feq) onde de é o vetor de deslocamento do elemento

In [15]:
# Extrair de1 do vetor da estrutura
de1 = d[0:4]

# Executar operação
fe1 = np.matmul(ke1, de1) - feq1

print(fe1)

# Extrair de1 do vetor da estrutura
de2 = d[2:6]

# Executar operação
fe2 = np.matmul(ke2, de2) - feq2

print(fe2)

[[  6063.82978723]
 [     0.        ]
 [  9936.17021277]
 [-15489.36170213]]
[[13286.80851064]
 [15489.36170213]
 [ 6713.19148936]
 [-9055.31914894]]


![Alt text](image-10.png)

![Alt text](image-11.png)